# Data Preprocssing

## Outlier Removal Before Filtering
Why: Removing outliers before bandpass filtering prevents filter artifacts and ensures the filtering process operates on clean data.
Method: The remove_outliers function detects outliers using Z-score thresholding and replaces them with interpolated values.

## Zero-Phase Filtering
Implementation: Replaced lfilter with filtfilt in the butter_bandpass_filter function.
Benefit: Zero-phase filtering avoids phase distortions that can be introduced by standard filtering methods, preserving the temporal characteristics of the EEG signals.

## Differential Entropy Computation
Adjustment: Added a small constant (1e-6) to the variance in compute_DE to prevent mathematical errors due to zero variance.
Importance: Ensures that DE values are computed accurately without encountering logarithm of zero.

## Data Reshaping to 10-20 System
Purpose: Aligns the processed data with the standard 10-20 EEG electrode placement system.
Manual Mapping: Channels are manually assigned to positions on an 8x9 grid. This requires careful attention to ensure accurate representation.

## Data Segregation
Trial Groups: Data and labels are divided into two groups (trials 1-16 and 17-24) for separate analysis or validation.
Consistency: Ensures that labels are correctly assigned to each segment based on the trial and participant.

In [3]:
import os
import sys
import math
import numpy as np
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, filtfilt  # Changed lfilter to filtfilt for zero-phase filtering
from scipy.io import loadmat

# Define label sets for different types of mat files
label_sets = {
    "type_1": [1, 2, 3, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 3, 2, 2, 3, 3, 0, 3, 0, 3],
    "type_2": [2, 1, 3, 0, 0, 2, 0, 2, 3, 3, 2, 3, 2, 0, 1, 1, 2, 1, 0, 3, 0, 1, 3, 1],
    "type_3": [1, 2, 2, 1, 3, 3, 3, 1, 1, 2, 1, 0, 2, 3, 3, 0, 2, 3, 0, 0, 2, 0, 1, 0]
}

# Function to remove outliers from a signal
def remove_outliers(signal, method='zscore', threshold=3):
    if method == 'zscore':
        mean = np.mean(signal)
        std = np.std(signal)
        z_scores = np.abs((signal - mean) / std)
        mask = z_scores < threshold
        signal_clean = signal.copy()
        # Replace outliers with interpolated values
        indices = np.arange(len(signal))
        signal_clean[~mask] = np.interp(indices[~mask], indices[mask], signal_clean[mask])
    elif method == 'mad':
        median = np.median(signal)
        mad = np.median(np.abs(signal - median))
        mask = np.abs(signal - median) < threshold * mad
        signal_clean = signal.copy()
        indices = np.arange(len(signal))
        signal_clean[~mask] = np.interp(indices[~mask], indices[mask], signal_clean[mask])
    else:
        raise ValueError("Method not recognized.")
    return signal_clean

# Function to create bandpass filter coefficients
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Function to apply bandpass filter to data using zero-phase filtering
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)  # Use filtfilt for zero-phase filtering
    return y

# Function to compute Differential Entropy (DE) of a signal
def compute_DE(signal):
    variance = np.var(signal, ddof=1) + 1e-6  # Add small constant to prevent log(0)
    return 0.5 * np.log(2 * np.pi * np.e * variance)  # Return DE value

# Function to decompose EEG data into different frequency bands and extract DE features
def decompose(file, name, label_set):
    # Load the .mat file containing the EEG data
    data = loadmat(file)
    frequency = 200  # Sampling rate of the SEED dataset is downsampled to 200Hz

    # Create empty arrays to store DE features and labels for both trial sets
    decomposed_de_1_16 = np.empty([0, 62, 5])
    decomposed_de_17_24 = np.empty([0, 62, 5])
    label_1_16 = np.array([])
    label_17_24 = np.array([])

    # Loop through all 24 trials in the dataset
    for trial in range(24):
        # Load the EEG data for the current trial
        tmp_trial_signal = data[name + '_eeg' + str(trial + 1)]
        # Number of samples per segment (0.5 seconds per segment, with a sampling rate of 200Hz)
        num_sample = int(len(tmp_trial_signal[0]) / 100)
        print('{}-{}'.format(trial + 1, num_sample))

        # Initialize temporary array to store DE features for each channel
        temp_de = np.empty([0, num_sample])
        # Assign labels for each sample in the current trial
        if trial < 16:
            label_1_16 = np.append(label_1_16, [label_set[trial]] * num_sample)
        else:
            label_17_24 = np.append(label_17_24, [label_set[trial]] * num_sample)

        # Loop through each channel (total 62 channels)
        for channel in range(62):
            trial_signal = tmp_trial_signal[channel]

            # Remove outliers before filtering
            trial_signal = remove_outliers(trial_signal, method='zscore', threshold=3)

            # Apply bandpass filters to extract different frequency bands
            delta = butter_bandpass_filter(trial_signal, 1, 4, frequency, order=3)
            theta = butter_bandpass_filter(trial_signal, 4, 8, frequency, order=3)
            alpha = butter_bandpass_filter(trial_signal, 8, 14, frequency, order=3)
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            gamma = butter_bandpass_filter(trial_signal, 31, 51, frequency, order=3)

            # Initialize arrays to store DE values for each frequency band
            DE_delta = np.zeros(shape=[0], dtype=float)
            DE_theta = np.zeros(shape=[0], dtype=float)
            DE_alpha = np.zeros(shape=[0], dtype=float)
            DE_beta = np.zeros(shape=[0], dtype=float)
            DE_gamma = np.zeros(shape=[0], dtype=float)

            # Compute DE features for each frequency band in each segment
            for index in range(num_sample):
                start_idx = index * 100
                end_idx = (index + 1) * 100
                DE_delta = np.append(DE_delta, compute_DE(delta[start_idx:end_idx]))
                DE_theta = np.append(DE_theta, compute_DE(theta[start_idx:end_idx]))
                DE_alpha = np.append(DE_alpha, compute_DE(alpha[start_idx:end_idx]))
                DE_beta = np.append(DE_beta, compute_DE(beta[start_idx:end_idx]))
                DE_gamma = np.append(DE_gamma, compute_DE(gamma[start_idx:end_idx]))

            # Stack the DE features for each frequency band
            temp_de = np.vstack([temp_de, DE_delta])
            temp_de = np.vstack([temp_de, DE_theta])
            temp_de = np.vstack([temp_de, DE_alpha])
            temp_de = np.vstack([temp_de, DE_beta])
            temp_de = np.vstack([temp_de, DE_gamma])

        # Reshape the DE features to match the desired format
        temp_trial_de = temp_de.reshape(-1, 5, num_sample)
        temp_trial_de = temp_trial_de.transpose([2, 0, 1])  # Rearrange dimensions to match desired format

        # Stack trial data into the appropriate group
        if trial < 16:
            decomposed_de_1_16 = np.vstack([decomposed_de_1_16, temp_trial_de])
        else:
            decomposed_de_17_24 = np.vstack([decomposed_de_17_24, temp_trial_de])

    return decomposed_de_1_16, label_1_16, decomposed_de_17_24, label_17_24

# Main script to extract features and save data
file_path = 'D:/BigData/SEED_IV/SEED_IV/eeg_raw_data/'

# List of participant names and short names used in file naming
people_name = ['1_20160518', '1_20161125', '1_20161126',
               '2_20150915', '2_20150920', '2_20151012',
               '3_20150919', '3_20151018', '3_20151101',
               '4_20151111', '4_20151118', '4_20151123',
               '5_20160406', '5_20160413', '5_20160420',
               '6_20150507', '6_20150511', '6_20150512',
               '7_20150715', '7_20150717', '7_20150721',
               '8_20151103', '8_20151110', '8_20151117',
               '9_20151028', '9_20151119', '9_20151209',
               '10_20151014', '10_20151021', '10_20151023',
               '11_20150916', '11_20150921', '11_20151011',
               '12_20150725', '12_20150804', '12_20150807',
               '13_20151115', '13_20151125', '13_20161130',
               '14_20151205', '14_20151208', '14_20151215',
               '15_20150508', '15_20150514', '15_20150527']

short_name = ['cz', 'cz', 'cz',
              'ha', 'ha', 'ha',
              'hql', 'hql', 'hql',
              'ldy', 'ldy', 'ldy',
              'ly', 'ly', 'ly',
              'mhw', 'mhw', 'mhw',
              'mz', 'mz', 'mz',
              'qyt', 'qyt', 'qyt',
              'rx', 'rx', 'rx',
              'tyc', 'tyc', 'tyc',
              'whh', 'whh', 'whh',
              'wll', 'wll', 'wll',
              'wq', 'wq', 'wq',
              'zjd', 'zjd', 'zjd',
              'zjy', 'zjy', 'zjy']

# Initialize empty arrays for storing the final DE features and labels for both trial groups
X_1_16 = np.empty([0, 62, 5])
y_1_16 = np.empty([0])
X_17_24 = np.empty([0, 62, 5])
y_17_24 = np.empty([0])

# Loop through all participants to extract DE features
for i in range(len(people_name)):  # Loop through all 45 experiments (15 participants, 3 trials each)
    file_name = os.path.join(file_path, people_name[i])
    print('processing {}'.format(people_name[i]))

    # Select the appropriate label set based on the type of file
    if i % 3 == 0:
        label_set = label_sets["type_1"]
    elif i % 3 == 1:
        label_set = label_sets["type_2"]
    else:
        label_set = label_sets["type_3"]

    decomposed_de_1_16, label_1_16, decomposed_de_17_24, label_17_24 = decompose(file_name, short_name[i], label_set)

    # Stack the features and labels for the two trial groups
    X_1_16 = np.vstack([X_1_16, decomposed_de_1_16])
    y_1_16 = np.append(y_1_16, label_1_16)
    X_17_24 = np.vstack([X_17_24, decomposed_de_17_24])
    y_17_24 = np.append(y_17_24, label_17_24)

# Create output directory if it doesn't exist
output_dir = "D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/"
os.makedirs(output_dir, exist_ok=True)

# Save the extracted DE features and labels as .npy files
np.save(os.path.join(output_dir, "X_1_16.npy"), X_1_16)
np.save(os.path.join(output_dir, "y_1_16.npy"), y_1_16)
np.save(os.path.join(output_dir, "X_17_24.npy"), X_17_24)
np.save(os.path.join(output_dir, "y_17_24.npy"), y_17_24)

# Load the saved features and labels
X_1_16 = np.load(os.path.join(output_dir, 'X_1_16.npy'))
y_1_16 = np.load(os.path.join(output_dir, 'y_1_16.npy'))
X_17_24 = np.load(os.path.join(output_dir, 'X_17_24.npy'))
y_17_24 = np.load(os.path.join(output_dir, 'y_17_24.npy'))

def reshape(y, X):
    # Reshape the 62-channel data into an 8x9 matrix (based on the 10-20 electrode system)
    X89 = np.zeros((len(y), 8, 9, 5))  # Create an empty array for the 8x9x5 data
    X89[:, 0, 2, :] = X[:, 3, :]  # Assign values to specific positions in the 8x9 matrix
    X89[:, 0, 3:6, :] = X[:, 0:3, :]
    X89[:, 0, 6, :] = X[:, 4, :]

    # Assign values for the middle rows of the 8x9 matrix
    for i in range(5):
        X89[:, i + 1, :, :] = X[:, 5 + i * 9:5 + (i + 1) * 9, :]

    # Assign values for the last two rows of the 8x9 matrix
    X89[:, 6, 1:8, :] = X[:, 50:57, :]
    X89[:, 7, 2:7, :] = X[:, 57:62, :]
    return X89

X89_1_16 = reshape(y_1_16, X_1_16)
X89_17_24 = reshape(y_17_24, X_17_24)

# Save the reshaped 8x9 matrix data
np.save(os.path.join(output_dir, "X89_1_16.npy"), X89_1_16)
np.save(os.path.join(output_dir, "X89_17_24.npy"), X89_17_24)


processing 1_20160518
1-336
2-190
3-398
4-260
5-176
6-324
7-306
8-418
9-290
10-338
11-100
12-220
13-434
14-338
15-518
16-282
17-136
18-358
19-280
20-96
21-224
22-224
23-350
24-274
processing 1_20161125
1-442
2-202
3-278
4-292
5-428
6-222
7-278
8-368
9-276
10-166
11-480
12-100
13-292
14-216
15-352
16-122
17-374
18-392
19-364
20-86
21-298
22-352
23-196
24-152
processing 1_20161126
1-340
2-260
3-184
4-364
5-386
6-212
7-516
8-186
9-208
10-128
11-414
12-330
13-314
14-154
15-230
16-354
17-114
18-140
19-366
20-178
21-318
22-310
23-330
24-312
processing 2_20150915
1-336
2-190
3-398
4-260
5-176
6-324
7-306
8-418
9-290
10-338
11-100
12-220
13-434
14-338
15-518
16-282
17-136
18-358
19-280
20-96
21-224
22-224
23-350
24-274
processing 2_20150920
1-442
2-202
3-278
4-292
5-428
6-222
7-278
8-368
9-276
10-166
11-480
12-100
13-292
14-216
15-352
16-122
17-374
18-392
19-364
20-86
21-298
22-352
23-196
24-152
processing 2_20151012
1-340
2-260
3-184
4-364
5-386
6-212
7-516
8-186
9-208
10-128
11-414
12-330
13

## Loading and Reshaping Data
Purpose: Load preprocessed EEG data and reshape it to group data by participants.
Implementation: Loaded data from .npy files (X89_1_16.npy, X89_17_24.npy) and reshaped them into arrays of shape (15, num_samples_per_participant, 8, 9, 5) to organize the data by the 15 participants.

## Defining Segment Length
Segment Length (t): Set to 6, which corresponds to a 3-second segment (since each time step represents 0.5 seconds).
Purpose: Capture temporal dependencies over meaningful durations in the EEG data for time-series analysis.

## Segmenting Data into Fixed-Length Sequences
Function: segment_data(falx, t, lengths, labels)
Purpose: Segment the data into fixed-length sequences and assign corresponding labels.
Method:
- Calculated trial boundaries using cumulative sums of the trial lengths provided in lengths.
- Iterated over participants (nb) and trials (j), extracting non-overlapping segments of length t.
- Managed indices carefully to ensure correct segment extraction without overlap.
- Assigned labels to each segment based on the trial's label from the labels list.
- Resized the pre-allocated array new_x to exclude unused space after segmentation.

## Defining Trial Lengths and Labels
Purpose: Provide necessary information for accurate segmentation and label assignment.
Data:
lengths_1_16 and lengths_17_24: Lists containing the number of time steps for each trial in datasets 1-16 and 17-24, respectively.
all_label_1_16 and all_label_17_24: Lists containing labels corresponding to each trial in the respective datasets.

## Segmenting and Saving Data
Process:
- Applied the segment_data function to both datasets:
  - For trials 1-16: x_89_Seg_1_16, y_89_Seg_1_16 were obtained from falx_1_16.
  - For trials 17-24: x_89_Seg_17_24, y_89_Seg_17_24 were obtained from falx_17_24.
- Printed shapes of the resulting segmented data and labels to verify correctness.

Saving:
- Saved the segmented data and labels to .npy files:
  - Segmented data: t6x_89_Seg_1_16.npy, t6x_89_Seg_17_24.npy.
  - Labels: t6y_89_Seg_1_16.npy, t6y_89_Seg_17_24.npy.




In [10]:
# segements
import numpy as np

X89_1_16 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/X89_1_16.npy")
print('{}-{}'.format('x_train shape', X89_1_16.shape))#x_train shape-(210330, 8, 9, 5)
img_rows, img_cols, num_chan = 8, 9, 5
falx_1_16 = X89_1_16
falx_1_16 = falx_1_16.reshape((15, int(X89_1_16.shape[0] / 15), img_rows, img_cols, num_chan)) #falx_1_16 shape-(15, 14022, 8, 9, 5)
print('{}-{}'.format('falx_1_16 shape', falx_1_16.shape))

X89_17_24 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/X89_17_24.npy")
print('{}-{}'.format('x_train shape', X89_17_24.shape))#x_train shape-(93360, 8, 9, 5)
img_rows, img_cols, num_chan = 8, 9, 5
falx_17_24 = X89_17_24
falx_17_24 = falx_17_24.reshape((15, int(X89_17_24.shape[0] / 15), img_rows, img_cols, num_chan)) #falx_17_24 shape-(15, 6224, 8, 9, 5)
print('{}-{}'.format('falx_17_24 shape', falx_17_24.shape))

t = 6 #(0.5s ->3s segement  6 pieces)

def segment_data(falx, t, lengths, labels):
  """Segments data into fixed-length segments with corresponding labels.

  Args:
    falx: The input data array.
    t: The length of each segment.
    lengths: A list of lengths for each segment.
    labels: A list of labels corresponding to each segment.

  Returns:
    new_x: The segmented data array.
    new_y: The corresponding label array.
  """

  # Calculate boundaries from lengths
  boundaries = np.cumsum(lengths)
  print('{}-{}'.format('boundaries', boundaries))

  total_segments = 100000 #pre-allocated number
  print('{}-{}'.format('total_segments', total_segments))

  # Pre-allocate new_x with correct dimensions
  new_x = np.empty([falx.shape[0], total_segments, t, 8, 9, 5])  
  new_y = np.array([])

  for nb in range(falx.shape[0]):
    z = 0
    i = 0
    for j, bound in enumerate(boundaries):
      while i + t <= bound:
        # Assign segments directly, taking all 6 time steps at once
        new_x[nb, z] = falx[nb, i:i + t]  # Assign to the first t indices of the segment
        new_y = np.append(new_y, labels[j])
        i = i + t
        z = z + 1
      i = bound
    print('{}-{}'.format(nb, z))
    # Resize new_x to exclude empty cells (segments that were not filled)
    new_x = new_x[:, :z, :, :, :, :]
  return new_x, new_y

lengths_1_16 =[336,190,398,260,176,324,306,418,290,338,100,220,434,338,518,282,
          442, 202, 278,292,428,222,278, 368, 276, 166, 480, 100, 292, 216, 352, 122, 
          340, 260, 184, 364, 386, 212, 516, 186, 208, 128, 414, 330, 314, 154, 230, 354]
lengths_17_24 =[136,358,280,96,224,224,350,274, 
          374, 392, 364, 86, 298, 352, 196, 152,
          114, 140, 366, 178, 318, 310, 330, 312]
all_label_1_16 = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,
             2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,
             1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0]
all_label_17_24 = [2,2,3,3,0,3,0,3,
                   2,1,0,3,0,1,3,1,
                   2,3,0,0,2,0,1,0]

x_89_Seg_1_16, y_89_Seg_1_16 = segment_data(falx_1_16, 6, lengths_1_16, all_label_1_16)
x_89_Seg_17_24, y_89_Seg_17_24 = segment_data(falx_17_24, 6, lengths_17_24, all_label_17_24)

print('{}-{}'.format('x_89_Seg_1_16 shape', x_89_Seg_1_16.shape))
print('{}-{}'.format('y_89_Seg_1_16 shape', y_89_Seg_1_16.shape))
print('{}-{}'.format('x_89_Seg_17_24 shape', x_89_Seg_17_24.shape))
print('{}-{}'.format('y_89_Seg_17_24 shape', y_89_Seg_17_24.shape))

np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t'+str(t)+'x_89_Seg_1_16.npy', x_89_Seg_1_16)#(15, 2320, 6, 8, 9, 5)
np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t'+str(t)+'y_89_Seg_1_16.npy', y_89_Seg_1_16)#(34800,)
np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t'+str(t)+'x_89_Seg_17_24.npy', x_89_Seg_17_24)#(15, 1028, 6, 8, 9, 5)
np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t'+str(t)+'y_89_Seg_17_24.npy', y_89_Seg_17_24)#(15420,)


x_train shape-(210330, 8, 9, 5)
falx_1_16 shape-(15, 14022, 8, 9, 5)
x_train shape-(93360, 8, 9, 5)
falx_17_24 shape-(15, 6224, 8, 9, 5)
boundaries-[  336   526   924  1184  1360  1684  1990  2408  2698  3036  3136  3356
  3790  4128  4646  4928  5370  5572  5850  6142  6570  6792  7070  7438
  7714  7880  8360  8460  8752  8968  9320  9442  9782 10042 10226 10590
 10976 11188 11704 11890 12098 12226 12640 12970 13284 13438 13668 14022]
total_segments-100000
0-2320
1-2320
2-2320
3-2320
4-2320
5-2320
6-2320
7-2320
8-2320
9-2320
10-2320
11-2320
12-2320
13-2320
14-2320
boundaries-[ 136  494  774  870 1094 1318 1668 1942 2316 2708 3072 3158 3456 3808
 4004 4156 4270 4410 4776 4954 5272 5582 5912 6224]
total_segments-100000
0-1028
1-1028
2-1028
3-1028
4-1028
5-1028
6-1028
7-1028
8-1028
9-1028
10-1028
11-1028
12-1028
13-1028
14-1028
x_89_Seg_1_16 shape-(15, 2320, 6, 8, 9, 5)
y_89_Seg_1_16 shape-(34800,)
x_89_Seg_17_24 shape-(15, 1028, 6, 8, 9, 5)
y_89_Seg_17_24 shape-(15420,)


# Model Traning and Evaluation

## Objective: Train a deep learning model for emotion recognition from EEG data using a combination of CNN, Transformer, and GRU layers.

## Data Handling:

Loaded and normalized EEG data.
Prepared data for input into the model by reshaping and selecting specific frequency bands.
## Model Design:

Created a base CNN with residual blocks for feature extraction.
Integrated a Transformer encoder to capture temporal dependencies.
Added a Bidirectional GRU layer for sequential modeling.
Used a softmax output layer for classification into four emotion classes.

## Training and Evaluation:

Used K-Fold cross-validation to ensure robustness.
Employed early stopping and learning rate reduction for efficient training.
Evaluated model performance on both test and validation sets.

## Results:

Calculated mean and standard deviation of accuracies.
Saved trained models for future use.

### Data Partitioning and Experimental Setup
#### Training and Validation Data (Trials 1–16):

- The EEG data from trials 1–16 is used exclusively for training and validation.
  - To ensure robust model evaluation during the development phase, a 5-fold cross-validation (K-Fold) approach is applied:
  - The trials 1–16 portion of the dataset is split into 5 subsets (folds).
- For each fold:
  - One subset is used as the validation set.
  - The remaining four subsets are combined to form the training set.
  - This process is repeated 5 times, each time with a different subset serving as the validation set.
  - The goal of this step is to fine-tune hyperparameters, assess model stability, and reduce overfitting by examining performance across multiple splits.

#### Dedicated Test Set (Trials 17–24):

- After the model has been trained and validated using the 1–16 trial data, a completely separate set of trials (17–24) is used as a final test set.
- This test set is not involved in any way during the training or validation phases.
- Evaluating the model on trials 17–24 provides an unbiased estimate of the model’s generalization performance on new, unseen data.

In [2]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Concatenate, Reshape, BatchNormalization, Bidirectional, GRU, Add, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from keras.models import  Model
from keras.callbacks import ReduceLROnPlateau
import keras.backend as K
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Layer,Lambda
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM,BatchNormalization,Dropout
from keras.models import Model

import keras
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.config.list_physical_devices('GPU'))
from keras import backend as K
import time
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping
import datetime

# Set GPU configuration
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.config.list_physical_devices('GPU'))

# Data Loading and Reshaping
num_classes = 4
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

#==============================load data=================================================

x_89_Seg_1_16 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t6x_89_Seg_1_16.npy")
y_89_Seg_1_16 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t6y_89_Seg_1_16.npy")
x_89_Seg_17_24 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t6x_89_Seg_17_24.npy")
y_89_Seg_17_24 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session1_2_3_exp1/t6y_89_Seg_17_24.npy")

def normalization(y,falx):
    one_y = to_categorical(y, num_classes)
    print('{}-{}'.format('one_y categorical shape', one_y.shape))

    one_falx_1 = falx.reshape((-1, 6, img_rows, img_cols, 5))  # reshape each person's segments
    one_falx = one_falx_1[:, :, :, :, 1:5]  # only 4 bands since last band reflects sleep feature
    return one_falx, one_y

one_x_1_16, one_y_1_16 = normalization(y_89_Seg_1_16,x_89_Seg_1_16)
one_x_17_24, one_y_17_24 = normalization(y_89_Seg_17_24,x_89_Seg_17_24)

#======================model=============================================================
# Base CNN Network with Residual Blocks
def create_base_network(input_dim):
    input_layer = Input(shape=input_dim)
    x = Conv2D(32, 5, activation='relu', padding='same', name='conv1')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # First Residual Block
    residual = Conv2D(128, 4, activation='relu', padding='same', name='conv2')(x)
    residual = BatchNormalization()(residual)
    residual = Dropout(0.2)(residual)
    x = Conv2D(128, 4, activation='relu', padding='same', name='conv2_residual')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Add()([x, residual])
    
    # Second Residual Block
    residual = Conv2D(256, 4, activation='relu', padding='same', name='conv3')(x)
    residual = BatchNormalization()(residual)
    residual = Dropout(0.2)(residual)
    x = Conv2D(256, 4, activation='relu', padding='same', name='conv3_residual')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Add()([x, residual])
    
    x = Conv2D(64, 1, activation='relu', padding='same', name='conv4')(x)
    x = MaxPooling2D(2, 2, name='pool1')(x)
    x = Flatten(name='fla1')(x)
    x = Dense(256, activation='relu', name='dense1')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Reshape((1, 256), name='reshape')(x)
    
    return Model(inputs=input_layer, outputs=x)

# Vision Transformer Encoder Block
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    # Multi-Head Self Attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attn_output = Dropout(rate)(attn_output)
    # Add & Norm
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    # Feed Forward Network
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(embed_dim)(ffn_output)
    ffn_output = Dropout(rate)(ffn_output)
    # Add & Norm
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Training and Evaluation

all_acc = []
all_val= []

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

K.clear_session()
start = time.time()

# Use one_x_17_24 and one_y_17_24 as the validation set
X_val, y_val = one_x_17_24, one_y_17_24

# Split one_x_1_16 and one_y_1_16 into train and test sets (7:3 ratio)
X_train_all, X_test, y_train_all, y_test = train_test_split(one_x_1_16, one_y_1_16, test_size=0.3, random_state=42, stratify=one_y_1_16.argmax(1))

for fold, (train_indices, val_indices) in enumerate(kf.split(X_train_all)):
    # Split into current fold's train and validation sets
    X_fold_train, X_fold_val = X_train_all[train_indices], X_train_all[val_indices]
    y_fold_train, y_fold_val = y_train_all[train_indices], y_train_all[val_indices]

    img_size = (img_rows, img_cols, num_chan)

    # Create base network
    base_network = create_base_network(img_size)
                        
    inputs = [Input(shape=img_size) for _ in range(6)]
    out_all = Concatenate(axis=1)([base_network(inp) for inp in inputs])

    # Transformer Encoder Block
    embed_dim = 256
    num_heads = 4
    ff_dim = 512
    transformer_output = transformer_encoder(out_all, embed_dim, num_heads, ff_dim)

    # Bidirectional GRU Layer
    bidir_gru = Bidirectional(GRU(128, return_sequences=False))(transformer_output)

    # Output Layer
    out_layer = Dense(4, activation='softmax', name='out')(bidir_gru)
    model = Model(inputs, out_layer)

    #tensorboard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0005),
                      metrics=['accuracy'])
    
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

    #tensorboard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Fit the model
    model.fit(
    [X_fold_train[:, i, :, :, :] for i in range(6)],
    y_fold_train,
    epochs=150,
    batch_size=batch_size,
    verbose=1,
    validation_data=([X_fold_val[:, i, :, :, :] for i in range(6)], y_fold_val),
    callbacks=[early_stopping, reduce_lr, tensorboard_callback])


    # Evaluate the model on the test set
    scores_test = model.evaluate(
    [X_test[:, i, :, :, :] for i in range(6)],
    y_test,
    verbose=0)
    
    # Evaluate the model on the dedicated validation set
    scores_val = model.evaluate(
    [X_val[:, i, :, :, :] for i in range(6)],
    y_val,
    verbose=0)
    
    #model.save(f'ResCNN-TransGRU_fold{fold + 1}.h5')
    
    print(f"Fold {fold + 1} - Test Accuracy: {scores_test[1] * 100:.2f}%, Validation Accuracy: {scores_val[1] * 100:.2f}%")
    all_acc.append(scores_test[1] * 100)
    all_val.append(scores_val[1] * 100)
    
print("acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))

print("val: {}".format(all_val))
print("mean vali: {}".format(np.mean(all_val)))
print("std vali: {}".format(np.std(all_val)))

end = time.time()
print("%.2f" % (end - start))  # Run time


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
one_y categorical shape-(34800, 4)
one_y categorical shape-(15420, 4)
Epoch 1/150
153/153 [==============================] - 16s 68ms/step - loss: 1.3026 - accuracy: 0.3975 - val_loss: 1.2538 - val_accuracy: 0.4368 - lr: 5.0000e-04
Epoch 2/150
153/153 [==============================] - 9s 60ms/step - loss: 1.1610 - accuracy: 0.4870 - val_loss: 1.1173 - val_accuracy: 0.5094 - lr: 5.0000e-04
Epoch 3/150
153/153 [==============================] - 9s 61ms/step - loss: 1.0676 - accuracy: 0.5380 - val_loss: 1.0155 - val_accuracy: 0.5782 - lr: 5.0000e-04
Epoch 4/150
153/153 [==============================] - 9s 61ms/step - loss: 0.9845 - accuracy: 0.5835 - val_loss: 1.0226 - val_accuracy: 0.5675 - lr: 5.0000e-04
Epoch 5/150
153/153 [==============================] - 9s 61ms/step - loss: 0.9185 - accuracy: 0.6170 - val_loss: 0.8932 - val_accuracy: 0.6361 - lr: 5

## Output

acc: [87.01149225234985, 87.5, 87.41379380226135, 87.51915693283081, 86.82950139045715]
- mean acc: 87.25478887557983
- std acc: 0.28119894161256664

val: [47.6199746131897, 47.34111428260803, 48.294422030448914, 48.68352711200714, 48.715952038764954]
- mean vali: 48.13099801540375
- std vali: 0.5584314191875352
